In [1]:
import cv2
import numpy as np
import glob
from cvxopt import matrix,solvers
from scipy.spatial.distance import cdist,pdist,squareform
import multiprocessing

from collections import defaultdict


# Part (a)

In [2]:
def preprocess(sample):
    sample =  (cv2.resize(sample, (16, 16)))/255.0
    return sample.flatten()

In [3]:
def read_sample(path):
    sample = cv2.imread(path)
    return preprocess(sample)

In [4]:
class0 = 1
class1 = 2

In [5]:
for iter, file in enumerate(glob.glob(f'./train/{class0}/*.jpg')):
    if iter == 0:
        print(file)
        X0 = read_sample(file)
    else:
        X0 = np.vstack((X0, read_sample(file)))

./train/1/4271.jpg


In [6]:
X0.shape

(2380, 768)

In [7]:
regulising_factor = 1

In [8]:
for iter, file in enumerate(glob.glob(f'./train/{class1}/*.jpg')):
    if iter == 0:
        X1 = read_sample(file)
        print(file)
    else:
        X1 = np.vstack((X1, read_sample(file)))

./train/2/21230.jpg


In [9]:
X1.shape

(2380, 768)

In [10]:
X0

array([[0.00784314, 0.13333333, 0.01960784, ..., 0.09019608, 0.11764706,
        0.06666667],
       [0.09019608, 0.26666667, 0.2745098 , ..., 0.11372549, 0.18431373,
        0.2       ],
       [0.50588235, 0.49411765, 0.4627451 , ..., 0.36078431, 0.37647059,
        0.41960784],
       ...,
       [0.39215686, 0.42745098, 0.38823529, ..., 0.01960784, 0.03921569,
        0.03137255],
       [0.10196078, 0.42745098, 0.37254902, ..., 0.06666667, 0.18039216,
        0.21568627],
       [0.02352941, 0.07843137, 0.09019608, ..., 0.05490196, 0.11764706,
        0.18431373]])

In [11]:
-1*X0

array([[-0.00784314, -0.13333333, -0.01960784, ..., -0.09019608,
        -0.11764706, -0.06666667],
       [-0.09019608, -0.26666667, -0.2745098 , ..., -0.11372549,
        -0.18431373, -0.2       ],
       [-0.50588235, -0.49411765, -0.4627451 , ..., -0.36078431,
        -0.37647059, -0.41960784],
       ...,
       [-0.39215686, -0.42745098, -0.38823529, ..., -0.01960784,
        -0.03921569, -0.03137255],
       [-0.10196078, -0.42745098, -0.37254902, ..., -0.06666667,
        -0.18039216, -0.21568627],
       [-0.02352941, -0.07843137, -0.09019608, ..., -0.05490196,
        -0.11764706, -0.18431373]])

In [12]:
only_X_data = np.vstack((X0, X1))

all_X = np.vstack((-1*X0, X1))
y = np.vstack((-1*np.ones((X0.shape[0], 1)), np.ones((X1.shape[0], 1))))

negative_test_cases = X0.shape[0]
positive_test_cases = X1.shape[0]
number_of_test_case = all_X.shape[0]

P = np.dot(all_X, all_X.T)
print(P.shape)

q = -1*np.ones((number_of_test_case, 1))
print(q.shape)
# q = q.T

# P = -1*P
# y

(4760, 4760)
(4760, 1)


In [13]:
G = np.vstack((-1*np.eye(number_of_test_case), np.eye(number_of_test_case)))
h = np.vstack((np.zeros((number_of_test_case, 1)), regulising_factor*np.ones((number_of_test_case, 1))))
print(G.shape, h.shape)

(9520, 4760) (9520, 1)


In [14]:
A =  np.hstack((-1*np.ones((1, negative_test_cases)), np.ones((1, positive_test_cases))))
print(A.shape)
b = np.zeros((1))

(1, 4760)


In [15]:
P = matrix(P, tc='d')
q = matrix(q, tc='d')
G = matrix(G, tc='d')
h = matrix(h, tc='d')
A = matrix(A, tc='d')
b = matrix(b, tc='d')

In [16]:
sol = solvers.qp(P,q,G,h,A,b)

     pcost       dcost       gap    pres   dres
 0: -1.0489e+03 -1.2518e+04  8e+04  3e+00  4e-11
 1: -6.8322e+02 -8.0216e+03  2e+04  5e-01  4e-11
 2: -4.8967e+02 -3.1466e+03  4e+03  1e-01  2e-11
 3: -4.0304e+02 -1.5654e+03  2e+03  5e-02  2e-11
 4: -3.7361e+02 -7.7703e+02  6e+02  1e-02  2e-11
 5: -3.8001e+02 -5.4972e+02  2e+02  4e-03  2e-11
 6: -3.9526e+02 -4.6864e+02  8e+01  1e-03  2e-11
 7: -4.0571e+02 -4.3399e+02  3e+01  2e-04  2e-11
 8: -4.1139e+02 -4.2094e+02  1e+01  1e-05  2e-11
 9: -4.1426e+02 -4.1682e+02  3e+00  3e-06  2e-11
10: -4.1514e+02 -4.1559e+02  5e-01  2e-13  2e-11
11: -4.1533e+02 -4.1537e+02  3e-02  2e-13  2e-11
12: -4.1535e+02 -4.1535e+02  7e-04  3e-13  2e-11
13: -4.1535e+02 -4.1535e+02  1e-05  2e-13  2e-11
Optimal solution found.


In [17]:
solution = np.array(sol['x'])

## 1(a)

In [18]:
number_of_supp_vectors = np.sum(solution > 1e-6)
print(number_of_supp_vectors)

671


In [19]:
solution_with_non_zero_alphas = solution.copy()
solution_with_non_zero_alphas[solution_with_non_zero_alphas < 1e-6] = 0

In [20]:
the_alphas = y*solution_with_non_zero_alphas
weights = np.dot(the_alphas.T, only_X_data).T
b = np.mean(y - np.dot(only_X_data,weights))

In [21]:
print("Weights: ", weights)
print("Bias: ", b)  

Weights:  [[ 0.33829301]
 [-0.26910453]
 [ 0.45842852]
 [ 1.22820353]
 [-0.41419856]
 [-0.62812013]
 [ 0.27813785]
 [-0.12548144]
 [ 0.12764424]
 [ 0.30380884]
 [-0.32458783]
 [-0.12996054]
 [ 0.40146463]
 [-0.45321825]
 [ 0.08664593]
 [ 0.03842593]
 [-0.45694275]
 [ 0.39767928]
 [-0.37417386]
 [-0.58534695]
 [ 0.74742418]
 [ 0.08005849]
 [-0.59831858]
 [ 0.4050871 ]
 [ 0.17143342]
 [-0.27094614]
 [ 0.27104399]
 [ 0.29055994]
 [-0.47397033]
 [ 0.13617692]
 [-0.15716963]
 [-0.58033581]
 [ 0.49376764]
 [-0.22674986]
 [-0.38627233]
 [ 0.36119057]
 [ 0.96171373]
 [-0.14109074]
 [-0.1724775 ]
 [ 1.22362283]
 [-0.40372132]
 [-0.25394422]
 [ 1.04778586]
 [-0.61972547]
 [-0.20429805]
 [ 0.13061751]
 [-0.32518196]
 [ 0.3474874 ]
 [ 0.92676973]
 [-0.09994707]
 [-0.56315033]
 [ 1.51370532]
 [-0.00866827]
 [-0.55076057]
 [ 0.21050943]
 [-0.39908061]
 [ 0.04385673]
 [-0.06427645]
 [-0.18405081]
 [-0.16802644]
 [ 0.48596055]
 [ 0.35493829]
 [-0.19524448]
 [ 0.03691451]
 [ 0.13738083]
 [-0.6458985 ]


In [22]:
correct = 0
total = 0

for iter, file in enumerate(glob.glob(f'./val/{class0}/*.jpg')):
    total += 1 
    validate_on = read_sample(file)
    # print(validate_on.shape)
    if np.dot(validate_on, weights) + b <= 0:
        correct += 1
        
for iter, file in enumerate(glob.glob(f'./val/{class1}/*.jpg')):
    total += 1 
    validate_on = read_sample(file)
    if np.dot(validate_on, weights) + b >= 0:
        correct += 1

## 1(b)

In [23]:
print(f"Accuracy: {(100*correct)/total} %")

Accuracy: 92.0 %


## 1(c)

In [24]:
# TODO: Part 1(c)

## 2(a)

In [25]:
gamma = 0.001

In [26]:
KERNEL_MATRIX = (np.outer(y,y))* (np.exp(-gamma * cdist(only_X_data, only_X_data, 'sqeuclidean')))

# v = pdist(only_X_data, 'sqeuclidean')
# KERNEL_MATRIX = np.exp(-1*gamma*squareform(v))

q = -1*np.ones((number_of_test_case, 1))
G = np.vstack((-1*np.eye(number_of_test_case), np.eye(number_of_test_case)))
h = np.vstack((np.zeros((number_of_test_case, 1)), regulising_factor*np.ones((number_of_test_case, 1))))
print(G.shape, h.shape)
A =  np.hstack((-1*np.ones((1, negative_test_cases)), np.ones((1, positive_test_cases))))
# print(A.shape)
b = np.zeros((1))


(9520, 4760) (9520, 1)


In [27]:
KERNEL_MATRIX = matrix(KERNEL_MATRIX, tc='d')
q = matrix(q, tc='d')
G = matrix(G, tc='d')
h = matrix(h, tc='d')
A = matrix(A, tc='d')
b = matrix(b, tc='d')


In [28]:
kernel_sol = solvers.qp(KERNEL_MATRIX,q,G,h,A,b)
# only_X_data

     pcost       dcost       gap    pres   dres
 0: -1.1860e+03 -1.2578e+04  7e+04  3e+00  2e-13
 1: -7.3522e+02 -8.1296e+03  1e+04  4e-01  3e-13
 2: -5.6455e+02 -1.6716e+03  1e+03  8e-03  2e-13
 3: -6.6510e+02 -1.3790e+03  7e+02  5e-03  2e-13
 4: -7.4937e+02 -1.1580e+03  4e+02  2e-03  2e-13
 5: -7.7440e+02 -1.1039e+03  3e+02  2e-03  2e-13
 6: -8.0472e+02 -1.0385e+03  2e+02  9e-04  2e-13
 7: -8.3295e+02 -9.7750e+02  1e+02  3e-04  2e-13
 8: -8.4698e+02 -9.5141e+02  1e+02  2e-04  2e-13
 9: -8.6050e+02 -9.2677e+02  7e+01  1e-04  2e-13
10: -8.7205e+02 -9.0692e+02  4e+01  5e-05  2e-13
11: -8.7866e+02 -8.9617e+02  2e+01  2e-05  2e-13
12: -8.8279e+02 -8.8974e+02  7e+00  6e-06  2e-13
13: -8.8527e+02 -8.8638e+02  1e+00  8e-07  3e-13
14: -8.8573e+02 -8.8578e+02  5e-02  2e-08  3e-13
15: -8.8576e+02 -8.8576e+02  1e-03  6e-10  3e-13
16: -8.8576e+02 -8.8576e+02  3e-05  1e-11  3e-13
Optimal solution found.


In [29]:
kernel_solution = np.array(kernel_sol['x'])

In [30]:
number_of_supp_vectors = np.sum(kernel_solution > 1e-6)
print(number_of_supp_vectors)

1096


In [31]:
kernel_solution

array([[2.01777784e-09],
       [1.32327817e-09],
       [9.99999998e-01],
       ...,
       [3.92286634e-09],
       [1.49205752e-09],
       [2.76835478e-09]])

In [32]:
def infer_on_gaussian(X_to_infer_on, X_used_for_training,alpha_i_s,y,gamma = 0.001):
    KERNEL_MATRIX = (np.exp(-gamma * cdist(X_used_for_training, X_used_for_training, 'sqeuclidean')))
    bias = np.mean(y - np.dot(KERNEL_MATRIX,alpha_i_s*y))
    
    KERNEL_TO_INFER = (np.exp(-gamma * cdist(X_used_for_training, X_to_infer_on, 'sqeuclidean')))
    predictions = np.dot(KERNEL_TO_INFER.T, (alpha_i_s*y)) + bias

    # predictions = [1 if pred >=0 else -1 for pred in predictions]
    return predictions

In [33]:
correct = 0
total = 0
infer_actual = []

for iter, file in enumerate(glob.glob(f'./val/{class0}/*.jpg')):
    total += 1 
    validate_on = read_sample(file)
    if iter == 0:
        X_to_infer_on = validate_on
    else:
        X_to_infer_on = np.vstack((X_to_infer_on, validate_on))
    infer_actual.append(-1)
        
for iter, file in enumerate(glob.glob(f'./val/{class1}/*.jpg')):
    total += 1 
    validate_on = read_sample(file)
    X_to_infer_on = np.vstack((X_to_infer_on, validate_on))
    infer_actual.append(1)

predictions = infer_on_gaussian(X_to_infer_on, only_X_data,kernel_solution,y,gamma = 0.001)
predictions = [1 if pred >=0 else -1 for pred in predictions]


for pred, actual in zip(predictions, infer_actual):
    if pred == actual:
        correct += 1

print(f"Accuracy: {(100*correct)/total} %")

Accuracy: 93.75 %


In [34]:
from sklearn import svm

ski_svm = svm.SVC(kernel='linear')
ski_svm.fit(np.vstack((X0, X1)), np.hstack((np.zeros((X0.shape[0])), np.ones((X1.shape[0])))))


SVC(kernel='linear')

In [35]:
support_vectors_per_class = ski_svm.n_support_

num_support_vectors_class_0 = support_vectors_per_class[0]
num_support_vectors_class_1 = support_vectors_per_class[1]

print(f"Number of support vectors for class 0: {num_support_vectors_class_0}")
print(f"Number of support vectors for class 1: {num_support_vectors_class_1}")

Number of support vectors for class 0: 346
Number of support vectors for class 1: 321


In [36]:
correct = 0
total = 0

for iter, file in enumerate(glob.glob(f'./val/{class0}/*.jpg')):
    total += 1 
    validate_on = read_sample(file)
    # print(ski_svm.predict([validate_on]))
    if ski_svm.predict([validate_on]) == 0:
        correct += 1
        
for iter, file in enumerate(glob.glob(f'./val/{class1}/*.jpg')):
    total += 1 
    validate_on = read_sample(file)
    if ski_svm.predict([validate_on]) == 1:
        correct += 1


In [37]:
print(f"Accuracy with SKLearn: {(100*correct)/total} %")


Accuracy with SKLearn: 92.0 %


In [38]:
ski_svm_gauss = svm.SVC(kernel='rbf', gamma=gamma)

ski_svm_gauss.fit(np.vstack((X0, X1)), np.hstack((np.zeros((X0.shape[0])), np.ones((X1.shape[0])))))

num_support_vectors = len(ski_svm_gauss.support_vectors_)
print(f"Number of Support Vectors: {num_support_vectors}")



Number of Support Vectors: 1087


In [39]:
correct = 0
total = 0

for iter, file in enumerate(glob.glob(f'./val/{class0}/*.jpg')):
    total += 1 
    validate_on = read_sample(file)
    # print(ski_svm.predict([validate_on]))
    if ski_svm_gauss.predict([validate_on]) == 0:
        correct += 1
        
for iter, file in enumerate(glob.glob(f'./val/{class1}/*.jpg')):
    total += 1 
    validate_on = read_sample(file)
    if ski_svm_gauss.predict([validate_on]) == 1:
        correct += 1


In [40]:
print(f"Accuracy with SKLearn on gaussian is: {(100*correct)/total} %")


Accuracy with SKLearn on gaussian is: 93.75 %


# Multi Class classification

In [41]:
classification_dict = {}

In [59]:
def classify_between_two(class0,class1):
    print("Classifying between ", class0, " and ", class1)
    for iter, file in enumerate(glob.glob(f'./train/{class0}/*.jpg')):
        if iter == 0:
            # print(file)
            X0 = read_sample(file)
        else:
            X0 = np.vstack((X0, read_sample(file)))
        
    for iter, file in enumerate(glob.glob(f'./train/{class1}/*.jpg')):
        if iter == 0:
            X1 = read_sample(file)
            print(file)
        else:
            X1 = np.vstack((X1, read_sample(file)))
    
    only_X_data = np.vstack((X0, X1))

    all_X = np.vstack((-1*X0, X1))
    y = np.vstack((-1*np.ones((X0.shape[0], 1)), np.ones((X1.shape[0], 1))))

    negative_test_cases = X0.shape[0]
    positive_test_cases = X1.shape[0]
    number_of_test_case = all_X.shape[0]

    gamma = 0.001
    
    KERNEL_MATRIX = (np.outer(y,y))* (np.exp(-gamma * cdist(only_X_data, only_X_data, 'sqeuclidean')))
    
    q = -1*np.ones((number_of_test_case, 1))
    G = np.vstack((-1*np.eye(number_of_test_case), np.eye(number_of_test_case)))
    h = np.vstack((np.zeros((number_of_test_case, 1)), regulising_factor*np.ones((number_of_test_case, 1))))
    A =  np.hstack((-1*np.ones((1, negative_test_cases)), np.ones((1, positive_test_cases))))
    b = np.zeros((1))
    
    
    KERNEL_MATRIX = matrix(KERNEL_MATRIX, tc='d')
    q = matrix(q, tc='d')
    G = matrix(G, tc='d')
    h = matrix(h, tc='d')
    A = matrix(A, tc='d')
    b = matrix(b, tc='d')
    
    kernel_sol = solvers.qp(KERNEL_MATRIX,q,G,h,A,b)
    
    kernel_solution = np.array(kernel_sol['x'])
    
    for iter, file in enumerate(glob.glob(f'./val/{class0}/*.jpg')):
        validate_on = read_sample(file)
        if iter == 0:
            X_to_infer_on = validate_on
        else:
            X_to_infer_on = np.vstack((X_to_infer_on, validate_on))
        infer_actual.append(-1)
            
    for iter, file in enumerate(glob.glob(f'./val/{class1}/*.jpg')):
        validate_on = read_sample(file)
        X_to_infer_on = np.vstack((X_to_infer_on, validate_on))
        infer_actual.append(1)
    
    predictions = infer_on_gaussian(X_to_infer_on, only_X_data,kernel_solution,y,gamma = 0.001)
    
    prediciton_iterator = 0
    
    for iter, file in enumerate(glob.glob(f'./val/{class0}/*.jpg')):
        if file not in classification_dict:
            classification_dict[file] = []
        
        # print(predictions[prediciton_iterator])
        classify_as = class1 if predictions[prediciton_iterator] >=0 else class0
        classification_dict[file].append((classify_as,predictions[prediciton_iterator][0]))
        prediciton_iterator += 1
    
    for iter, file in enumerate(glob.glob(f'./val/{class1}/*.jpg')):
        if file not in classification_dict:
            classification_dict[file] = []
        
        classify_as = class1 if predictions[prediciton_iterator] >=0 else class0
        classification_dict[file].append((classify_as,predictions[prediciton_iterator][0]))
        prediciton_iterator += 1



In [60]:
# for i in range(0,3):
#     for j in range(0,3):
#         if i < j:
#             classify_between_two(i,j)

classify_between_two(0,1)

Classifying between  0  and  1
./train/1/4271.jpg
     pcost       dcost       gap    pres   dres
 0: -1.9393e+03 -1.4696e+04  9e+04  4e+00  4e-13
 1: -1.2731e+03 -1.0480e+04  2e+04  4e-01  4e-13
 2: -1.1845e+03 -2.6743e+03  1e+03  6e-13  4e-13
 3: -1.4357e+03 -2.1032e+03  7e+02  6e-14  4e-13
 4: -1.5025e+03 -1.9941e+03  5e+02  2e-12  4e-13
 5: -1.5733e+03 -1.8712e+03  3e+02  1e-12  4e-13
 6: -1.6192e+03 -1.7957e+03  2e+02  6e-13  4e-13
 7: -1.6452e+03 -1.7549e+03  1e+02  5e-13  4e-13
 8: -1.6661e+03 -1.7225e+03  6e+01  3e-13  4e-13
 9: -1.6769e+03 -1.7068e+03  3e+01  1e-12  4e-13
10: -1.6857e+03 -1.6947e+03  9e+00  8e-13  4e-13
11: -1.6890e+03 -1.6904e+03  1e+00  1e-13  5e-13
12: -1.6896e+03 -1.6897e+03  7e-02  6e-13  5e-13
13: -1.6896e+03 -1.6896e+03  2e-03  2e-13  5e-13
14: -1.6896e+03 -1.6896e+03  3e-05  1e-12  5e-13
Optimal solution found.
[-0.25849697]
[0.92707663]
[-2.89444727]
[-1.65080284]
[-0.79981767]
[0.2138211]
[-2.42851631]
[-2.84277962]
[-2.73871044]
[-2.12557709]
[-2.62

## Making it parallel

In [61]:
def classify_between_two_parallel(class0,class1,result_dict):
    classification_dict = {}
    print("Classifying between ", class0, " and ", class1)
    for iter, file in enumerate(glob.glob(f'./train/{class0}/*.jpg')):
        if iter == 0:
            # print(file)
            X0 = read_sample(file)
        else:
            X0 = np.vstack((X0, read_sample(file)))
        
    for iter, file in enumerate(glob.glob(f'./train/{class1}/*.jpg')):
        if iter == 0:
            X1 = read_sample(file)
            print(file)
        else:
            X1 = np.vstack((X1, read_sample(file)))
    
    only_X_data = np.vstack((X0, X1))

    all_X = np.vstack((-1*X0, X1))
    y = np.vstack((-1*np.ones((X0.shape[0], 1)), np.ones((X1.shape[0], 1))))

    negative_test_cases = X0.shape[0]
    positive_test_cases = X1.shape[0]
    number_of_test_case = all_X.shape[0]
#     for j in range(0,3):
#         if i < j:
#             classify_between_two(i,j)X.shape[0]

    gamma = 0.001
    
    KERNEL_MATRIX = (np.outer(y,y))* (np.exp(-gamma * cdist(only_X_data, only_X_data, 'sqeuclidean')))
    
    q = -1*np.ones((number_of_test_case, 1))
    G = np.vstack((-1*np.eye(number_of_test_case), np.eye(number_of_test_case)))
    h = np.vstack((np.zeros((number_of_test_case, 1)), regulising_factor*np.ones((number_of_test_case, 1))))
    A =  np.hstack((-1*np.ones((1, negative_test_cases)), np.ones((1, positive_test_cases))))
    b = np.zeros((1))
    
    
    KERNEL_MATRIX = matrix(KERNEL_MATRIX, tc='d')
    q = matrix(q, tc='d')
    G = matrix(G, tc='d')
    h = matrix(h, tc='d')
    A = matrix(A, tc='d')
    b = matrix(b, tc='d')
    
    kernel_sol = solvers.qp(KERNEL_MATRIX,q,G,h,A,b)
    
    kernel_solution = np.array(kernel_sol['x'])
    
    for iter, file in enumerate(glob.glob(f'./val/{class0}/*.jpg')):
        validate_on = read_sample(file)
        if iter == 0:
            X_to_infer_on = validate_on
        else:
            X_to_infer_on = np.vstack((X_to_infer_on, validate_on))
        infer_actual.append(-1)
            
    for iter, file in enumerate(glob.glob(f'./val/{class1}/*.jpg')):
        validate_on = read_sample(file)
        X_to_infer_on = np.vstack((X_to_infer_on, validate_on))
        infer_actual.append(1)
    
    predictions = infer_on_gaussian(X_to_infer_on, only_X_data,kernel_solution,y,gamma = 0.001)
    
    prediciton_iterator = 0
    
    for iter, file in enumerate(glob.glob(f'./val/{class0}/*.jpg')):
        if file not in classification_dict:
            classification_dict[file] = []
        
        # print(predictions)
        
        classify_as = class1 if predictions[prediciton_iterator] >=0 else class0
        classification_dict[file].append((classify_as,predictions[prediciton_iterator][0]))
        prediciton_iterator += 1
    
    for iter, file in enumerate(glob.glob(f'./val/{class1}/*.jpg')):
        if file not in classification_dict:
            classification_dict[file] = []
        
        classify_as = class1 if predictions[prediciton_iterator] >=0 else class0
        classification_dict[file].append((classify_as,predictions[prediciton_iterator][0]))
        prediciton_iterator += 1
        
    
    result_dict[(class0, class1)] = classification_dict




In [62]:
num_classes = 6

class_pairs = [(i, j) for i in range(num_classes) for j in range(i+1, num_classes)]

manager = multiprocessing.Manager()
result_dict = manager.dict()

pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())

for class_pair in class_pairs:
    pool.apply_async(classify_between_two_parallel, args=(class_pair[0], class_pair[1], result_dict))

pool.close()
pool.join()

final_classification_dict = {}
for classification_dict in result_dict.values():
    for key, value in classification_dict.items():
        if key not in final_classification_dict:
            final_classification_dict[key] = []
        final_classification_dict[key].extend(value)

Classifying between Classifying between Classifying between    00 1   and  and  and   2 2
1

./train/1/4271.jpg
./train/2/21230.jpg
./train/2/21230.jpg
     pcost       dcost       gap    pres   dres
 0: -1.9393e+03 -1.4696e+04  9e+04  4e+00  4e-13
     pcost       dcost       gap    pres   dres
 0: -1.1860e+03 -1.2578e+04  7e+04  3e+00  2e-13
     pcost       dcost       gap    pres   dres
 0: -2.8766e+03 -1.4989e+04  8e+04  3e+00  6e-13
 1: -1.2731e+03 -1.0480e+04  2e+04  4e-01  4e-13
 1: -7.3522e+02 -8.1296e+03  1e+04  4e-01  3e-13
 1: -1.9324e+03 -1.0948e+04  1e+04  2e-01  7e-13
 2: -1.1845e+03 -2.6743e+03  1e+03  6e-13  4e-13
 2: -5.6455e+02 -1.6716e+03  1e+03  8e-03  2e-13
 2: -2.0564e+03 -3.5589e+03  2e+03  2e-02  5e-13
 3: -1.4357e+03 -2.1032e+03  7e+02  6e-14  4e-13
 3: -6.6510e+02 -1.3790e+03  7e+02  5e-03  2e-13
 3: -2.4060e+03 -2.9060e+03  5e+02  6e-03  6e-13
 4: -1.5025e+03 -1.9941e+03  5e+02  2e-12  4e-13
 4: -7.4937e+02 -1.1580e+03  4e+02  2e-03  2e-13
 4: -2.5331e+03 -2

In [64]:
final_labels = {}
for file, classifications in final_classification_dict.items():
    
    votes = {}
    
    for classify_as, score in classifications:
        if classify_as not in votes:
            votes[classify_as] = (0,0)
        
        number_of_vote, score_till_now = votes[classify_as]
        votes[classify_as] = (number_of_vote + 1, score_till_now + abs(score))
        
    max_votes = max(votes, key=lambda k: (votes[k][0], votes[k][1]))

    
    final_labels[file] = max_votes


In [65]:
len(final_labels)

600

In [72]:
correct = 0
total = 0

for i in range(0,num_classes):
    for iter, file in enumerate(glob.glob(f'./val/{i}/*.jpg')):
        total += 1
        if final_labels[file] == i:
            correct += 1
        

In [67]:
total

200

In [73]:
print(f"Accuracy for multi classification is {(100*correct)/total}%")

Accuracy for multi classification is 92.0%


In [69]:
X_multi_class = None
y = None

for i in range(0,num_classes):
    for iter, file in enumerate(glob.glob(f'./train/{i}/*.jpg')):
        if X_multi_class is None:
            X_multi_class = read_sample(file)
            y_multi_class = np.array([i])
        else:
            X_multi_class = np.vstack((X_multi_class, read_sample(file)))
            y_multi_class = np.vstack((y, np.array([i])))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
ski_svm_gauss = svm.SVC(kernel='rbf', gamma=gamma)

ski_svm_gauss.fit(X_multi_class,y_multi_class)

num_support_vectors = len(ski_svm_gauss.support_vectors_)
print(f"Number of Support Vectors: {num_support_vectors}")



In [ ]:
correct = 0
total = 0

for i in range(0,num_classes):
    for iter, file in enumerate(glob.glob(f'./train/{i}/*.jpg')):
        total += 1 
        validate_on = read_sample(file)
        if ski_svm_gauss.predict([validate_on]) == 0:
            correct += 1

In [ ]:
print(f"Accuracy with SKLearn on gaussian for multi class is: {(100*correct)/total} %")
